In [5]:
# BETTER WAY 03 - bytes, str, unicode

print(b'foo')
print(type(b'foo'))

import os

def to_str(bytes_or_str):
    if isinstance(bytes_or_str, bytes):
        value = bytes_or_str.decode('utf-8')
    else:
        value = bytes_or_str
    return value # str 인스턴스
repr(to_str(b'foo'))

b'foo'
<class 'bytes'>


"'foo'"

In [16]:
# orthogonality #
##################################

def cal(a,b,c):
    return a+b+c

def show(price):
    return f"$ {price}"

def final(a,b,c, fmt=str):
    return fmt(cal(a,b,c))

final(10, 0.2, 0.5, fmt=show)


<function __main__.kkk(a, b)>

In [12]:
def lastNmae_and_FirstName(*Nmaes):
    return [[name, *name[1:10]] for name in Nmaes]


lastNmae_and_FirstName('이천수', '산다라박')

[['이천수', '천', '수'], ['산다라박', '다', '라', '박']]

In [2]:
from datetime import datetime, timedelta

class Bucket:
    def __init__(self, period):
        self.period_delta = timedelta(seconds=period)
        # 한 번 채우면 사용할 수 있는 시간(다 새기까지)
        self.reset_time = datetime.now()
        # 양동이를 채운 최근 시간을 기록
        self.quota = 0
        # 투입물의 양을 기록(quota: 할당량)
        
        self.max_quota = 0
        self.quota_consumed = 0

    def __repr__(self):
        return "Bucket(max_quota={}, quota_consumed={})".format(self.max_quota, self.quota_consumed)


    # quota를 동적으로 계산해 확인한다.
    @property
    def quota(self):
        return self.max_quota - self.quota_consumed

    # quota에 동적으로 변화를 준다.
    @quota.setter
    def quota(self, amount):
        delta = self.max_quota - amount
        if amount == 0:
            # quota를 0으로 할당하는 것은 새 기간의 할당량을 리셋하는 것
            self.quota_consumed = 0
            self.max_quota = 0
        elif delta < 0:
            # 새 기간의 할당량을 채움
            assert self.quota_consumed == 0
            self.max_quota = amount
        else:
            # 기간 동안 할당량을 소비함.
            assert self.max_quota >= self.quota_consumed
            self.quota_consumed += delta
    
    
def fill(bucket, amount):
    now = datetime.now()
    # 내용물이 모두 빠져나갈 만큼 시간이 오래됐으면,
    # 물이 다 빠져나갔을 것이기 때문에 quota를 0으로 두고, 시간 리셋
    if now - bucket.reset_time > bucket.period_delta:
        bucket.quota = 0
        bucket.reset_time = now
    bucket.quota += amount
    
def deduct(bucket, amount):
    now = datetime.now()

    # 시간이 너무 지나 물이 다 떨어짐. 퍼낼 수 없다.
    if now - bucket.reset_time > bucket.period_delta:
        return False
    # 퍼낼 양이 지금 양동이의 물 양보다 많다.
    elif bucket.quota < amount:
        return False

    bucket.quota -= amount  # amount만큼 물을 퍼냄
    return True

bucket = Bucket(60)
fill(bucket, 100)
print(bucket)

if deduct(bucket, 99):
    print("Had 99 quota")
else:
    print("Now enough for 99 quota")
    print(bucket)

if deduct(bucket, 3):
    print("Had 3 quota")
else:
    print("Not enough for 3 quota")
    print(bucket)



Bucket(quota=100)
Had 99 quota
Not enough for 3 quota
Bucket(quota=1)


In [3]:
class Registor:
    def __init__(self, ohms):
        self.ohms = ohms
        self.voltage = 0
        self.current = 0


class BoundedRegistance(Registor):
    def __init__(self, ohms):
        super().__init__(ohms)

    @property
    def ohms(self):
        return self._ohms

    @ohms.setter
    def ohms(self, ohms):
        if ohms <= 0:
            raise ValueError("Ohms must be over 0.")
        self._ohms = ohms
        
r3 = BoundedRegistance(1e3)
r3.ohms = 0

ValueError: Ohms must be over 0.